In [5]:
# usual imports
import os
import sys
import time
import glob
import datetime
import sqlite3
import pandas as pd
import numpy as np # get it at: http://numpy.scipy.org/
# path to the Million Song Dataset subset (uncompressed)
# CHANGE IT TO YOUR LOCAL CONFIGURATION
# msd_subset_path='/mnt/snap/data/MSD/MillionSongSubset'
# msd_subset_data_path=os.path.join(msd_subset_path,'data')
msd_subset_addf_path=os.path.join('/mnt/snap','AdditionalFiles')
# assert os.path.isdir(msd_subset_path),'wrong path' # sanity check
# # path to the Million Song Dataset code
# # CHANGE IT TO YOUR LOCAL CONFIGURATION
# msd_code_path='./data/MSD/MSongsDB'
# assert os.path.isdir(msd_code_path),'wrong path' # sanity check
# # we add some paths to python so we can import MSD code
# # Ubuntu: you can change the environment variable PYTHONPATH
# # in your .bashrc file so you do not have to type these lines
# sys.path.append(os.path.join(msd_code_path,'PythonSrc') )

taste_profile_data_path="./data/train_triplets.txt"
# filtered_taste_profile_data_path="./data/FilteredEchoNestTasteProfileSubset.csv"

# assert os.path.isfile(taste_profile_data_path)

# imports specific to the MSD
# import hdf5_getters as GETTERS

from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.sql import Row

In [2]:
def reduce_taste_subset(path='./data/FullEchoNestTasteProfileSubset.txt', 
                        to_path='./data/TeenyTinyEchoNestTasteProfileSubset.csv', downsample=0.001):
    data = pd.read_csv(path, sep="\t", header=None)
    data.columns = ['user', 'song', 'play_count']
    data.astype({'user': np.str, 'song': np.str, 'play_count': np.int32})
    data.sample(frac=downsample).to_csv(to_path, index=False)
# the following function simply gives us a nice string for
# a time lag in seconds
def strtimedelta(starttime,stoptime):
    return str(datetime.timedelta(seconds=stoptime-starttime))

# Get full data set

In [2]:
data = pd.read_csv(taste_profile_data_path, sep="\t", header=None)
data.columns = ['user', 'song', 'play_count']
data = data.astype({'user': np.str, 'song': np.str, 'play_count': np.int32})

In [3]:
data.head()

,user,song,play_count
0,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOAKIMP12A8C130995,1
1,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOAPDEY12A81C210A9,1
2,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBBMDR12A8C13253B,2
3,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBFNSP12AF72A0E22,1
4,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBFOVM12A58A7D494,1


# Subset dataset by the tracks that we have in MSD

In [23]:
# Get the song ids which are also in the database. 
track_ids = []

# for song_id in data['song'].values:
# let's redo all this work in SQLite in a few seconds
t1 = time.time()
# connect to database to get the metadata from MSD
conn = sqlite3.connect(os.path.join(msd_subset_addf_path,
                                    'track_metadata.db'))

q = "SELECT DISTINCT song_id, track_id, artist_name, title FROM songs"
#     q += " WHERE song_id='" + song_id + "'"
res = conn.execute(q)
res = res.fetchall()
conn.close()
t2 = time.time()

song_to_track = {}
track_to_song = {}
for r in res:
    song_to_track[r[0]] = r[1]
    track_to_song[r[1]] = r[0]
    



In [26]:
msd_df = pd.DataFrame(data=res, columns=["song_id", "track_id", "artist_id", "song_name"])

In [9]:
a = time.time()
print "SOAPDEY12A81C210A9" in song_to_track
print(time.time() - a)

True
0.000231027603149


In [279]:
b = time.time()
print "SOAPDEY12A81C210A9" in song_ids_dict.values()
print(time.time() - b)

False
0.0225830078125


In [280]:
small_data = data.iloc[:1000000]
# print(small_data.shape)
not_included = {}
a = time.time()
def foo(x):
    if x not in song_ids_dict:
        if x not in not_included:
            not_included[x] = 1
        return False
    return True
    
filtered_data = data[data["song"].map(foo)]

# filtered_data = data[data["song"].map(lambda x: x in song_ids_dict)]
print(time.time() - a)

20.8473460674


# Store and filter by 100,000 most popular songs

In [10]:
song_popularity = data.drop(["user"], axis=1)
song_popularity = song_popularity.groupby(["song"]).sum()

In [11]:
song_popularity.size

384546

In [19]:
song_popularity_subset = song_popularity.sort_values(["play_count"], ascending=False).iloc[:100000]


In [21]:
song_popularity_subset.head(20)


,play_count
song,
SOBONKR12A58A7A7E0,726885
SOAUWYT12A81C206F1,648239
SOSXLTC12AF72A7F54,527893
SOFRQTD12A81C233C0,425463
SOEGIYH12A6D4FC0E3,389880
SOAXGDH12A8C13F8A1,356533
SONYKOW12AB01849C9,292642
SOPUCYA12A8C13A694,274627
SOUFTBI12AB0183F65,268353


In [27]:
song_popularity_subset_dict = {s: 1 for s in song_popularity_subset.index}

In [31]:
len(song_popularity_subset_dict)

100000

In [15]:
data_subset = data[data["song"].map(lambda x: x in song_popularity_subset)]

In [16]:
len(data_subset)

44774993

# subset msd

In [37]:
len(msd_df)

1000000

In [35]:
len(set(msd_df["song_id"]))

999056

In [32]:
subset_msd_df = msd_df[msd_df["song_id"].map(lambda x : x in song_popularity_subset_dict)]

In [39]:
import pickle
len(subset_msd_df)
with open("data/song_track_artist.pkl", "w") as f:
    pickle.dump(subset_msd_df, f)

### Stored filtered Taste data

In [281]:
filtered_data.to_csv(filtered_taste_profile_data_path, index=False)

# Train the WMF model

In [11]:
def tune_ALS(train_data, validation_data, maxIter, regParams, ranks):
    """
    grid search function to select the best model based on RMSE of
    validation data
    Parameters
    ----------
    train_data: spark DF with columns ['user', 'song', 'play_count']
    
    validation_data: spark DF with columns ['user', 'song', 'play_count']
    
    maxIter: int, max number of learning iterations
    
    regParams: list of float, one dimension of hyper-param tuning grid
    
    ranks: list of float, one dimension of hyper-param tuning grid
    
    Return
    ------
    The best fitted ALS model with lowest RMSE score on validation data
    """
    # initial
    min_error = float('inf')
    best_rank = -1
    best_regularization = 0
    best_model = None
    for rank in ranks:
        for reg in regParams:
            # get ALS model
            # Note we set cold start strategy to 'drop' to ensure we don't get NaN evaluation metrics
            als = ALS(userCol="user", itemCol="song", ratingCol="play_count", implicitPrefs=True, alpha=40).setMaxIter(maxIter).setRank(rank).setRegParam(reg).setColdStartStrategy("drop")
            # train ALS model
            model = als.fit(train_data)
            # evaluate the model by computing the RMSE on the validation data
            predictions = model.transform(validation_data)  # give Nan to data that hasn't been seen before. cold_start drops those
            evaluator = RegressionEvaluator(metricName="rmse",
                                            labelCol="play_count",
                                            predictionCol="prediction")
            rmse = evaluator.evaluate(predictions)
            print('{} latent factors and regularization = {}: '
                  'validation RMSE is {}'.format(rank, reg, rmse))
            if rmse < min_error:
                min_error = rmse
                best_rank = rank
                best_regularization = reg
                best_model = model
    print('\nThe best model has {} latent factors and '
          'regularization = {}'.format(best_rank, best_regularization))
    return best_model

In [45]:
filtered_data = data_subset #pd.read_csv(filtered_taste_profile_data_path)

In [46]:
user_mapping = {}
last_user = 0

def user_map(user):
    global user_mapping, last_user
    if user not in user_mapping:
        user_mapping[user] = last_user
        last_user += 1

a = filtered_data["user"].map(lambda x: user_map(x))

In [47]:
song_mapping = {}
last_song = 0

def song_map(song):
    global song_mapping, last_song
    if song not in song_mapping:
        song_mapping[song] = last_song
        last_song += 1

a = filtered_data["song"].map(lambda x: song_map(x))

In [48]:
prep_data = filtered_data.copy()
prep_data["user"] = prep_data["user"].map(lambda x: user_mapping[x])
prep_data["song"] = prep_data["song"].map(lambda x: song_mapping[x])

In [49]:
sc.setCheckpointDir("checkpoint/")
spark_data = spark.createDataFrame(prep_data)
(training, val) = spark_data.randomSplit([0.8, 0.2])

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.readRDDFromFile.
: java.lang.OutOfMemoryError: Java heap space
	at org.apache.spark.api.python.PythonRDD$.readRDDFromInputStream(PythonRDD.scala:188)
	at org.apache.spark.api.python.PythonRDD$.readRDDFromFile(PythonRDD.scala:175)
	at org.apache.spark.api.python.PythonRDD.readRDDFromFile(PythonRDD.scala)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:357)
	at py4j.Gateway.invoke(Gateway.java:282)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.

In [27]:
training.show()

Py4JJavaError: An error occurred while calling o59.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 0.0 failed 1 times, most recent failure: Lost task 0.0 in stage 0.0 (TID 0, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/usr/local/opt/spark/python/lib/pyspark.zip/pyspark/worker.py", line 267, in main
    ("%d.%d" % sys.version_info[:2], version))
Exception: Python in worker has different version 3.6 than that in driver 2.7, PySpark cannot run with different minor versions.Please check environment variables PYSPARK_PYTHON and PYSPARK_DRIVER_PYTHON are correctly set.

	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.handlePythonException(PythonRunner.scala:452)
	at org.apache.spark.api.python.PythonRunner$$anon$1.read(PythonRunner.scala:588)
	at org.apache.spark.api.python.PythonRunner$$anon$1.read(PythonRunner.scala:571)
	at org.apache.spark.api.python.

In [169]:
model = tune_ALS(training, val, 30, [1e-3], [50])

50 latent factors and regularization = 0.001: validation RMSE is 5.13091755092

The best model has 50 latent factors and regularization = 0.001


In [172]:
model.rank

50

In [24]:
predictions = model.transform(val)

In [28]:
predictions

Column<play_count>

In [173]:
user_factors = model.userFactors.toPandas()

In [179]:
user_factors.head()

,id,features
0,0,"[-0.240919277072, 0.103509902954, -0.176839962..."
1,10,"[-0.0564561896026, 0.0740611702204, -0.0296464..."
2,20,"[-0.690784394741, 0.113340988755, 0.2471711635..."
3,30,"[0.0872035175562, -0.0911120772362, -0.6268772..."
4,40,"[0.189205631614, 0.422972619534, 0.38837027549..."


In [174]:
song_factors = model.itemFactors.toPandas()

In [180]:
song_factors.head()

,id,features
0,0,"[-0.0201986338943, -0.00560505082831, 0.173121..."
1,10,"[-0.0209625754505, 0.0568268969655, 0.08685378..."
2,20,"[-0.00502466084436, -0.0324248373508, 0.043798..."
3,30,"[0.235187828541, 0.0487095415592, -0.077217929..."
4,40,"[0.00416562194005, -0.0257982704788, -0.068397..."


In [175]:
user_mapping_inverted = dict(map(reversed, user_mapping.items()))

In [176]:
song_mapping_inverted = dict(map(reversed, song_mapping.items()))

In [182]:
user_factors["user"] = user_factors["id"].map(lambda x: user_mapping_inverted[x])
song_factors["song"] = song_factors["id"].map(lambda x: song_mapping_inverted[x])

In [185]:
final_user_factors = user_factors.drop(columns=["id"])
final_song_factors = song_factors.drop(columns=["id"])

In [186]:
final_song_factors.head()

,features,song
0,"[-0.0201986338943, -0.00560505082831, 0.173121...",SOWEZSI12A81C21CE6
1,"[-0.0209625754505, 0.0568268969655, 0.08685378...",SOGJPMB12A8C13A9DB
2,"[-0.00502466084436, -0.0324248373508, 0.043798...",SOVTFUO12A6310D813
3,"[0.235187828541, 0.0487095415592, -0.077217929...",SOLRYQR12A670215BF
4,"[0.00416562194005, -0.0257982704788, -0.068397...",SOGLABR12A6D4F9585


In [187]:
final_user_factors.head()

,features,user
0,"[-0.240919277072, 0.103509902954, -0.176839962...",b80344d063b5ccb3212f76538f3d9e43d87dca9e
1,"[-0.0564561896026, 0.0740611702204, -0.0296464...",403b3b867fc71dfdcc12652f30e88bdc7ccd9aa4
2,"[-0.690784394741, 0.113340988755, 0.2471711635...",1fdf267d8e81db0bb69093460fdab8254e69a2de
3,"[0.0872035175562, -0.0911120772362, -0.6268772...",99b5f915dd56a0150eb91605e1f6109e1e80c9de
4,"[0.189205631614, 0.422972619534, 0.38837027549...",01655ae6bc52e29c9cd100a7dde4e9eeae5e4031


In [189]:
final_song_factors.shape

(3616, 2)

In [190]:
final_user_factors.shape

(365465, 2)

In [244]:
final_song_factors.to_csv("./data/song_factors.csv", index=False)
final_user_factors.to_csv("./data/user_factors.csv", index=False)

In [268]:
dict_user_factors = final_user_factors.set_index("user").to_dict()["features"]

In [267]:
dict_song_factors = final_song_factors.set_index("song").to_dict()["features"]

In [269]:
import pickle
with open("./data/user_factors.pkl", "w") as f:
    pickle.dump(dict_user_factors, f)

with open("./data/song_factors.pkl", "w") as f:
    pickle.dump(dict_song_factors, f)

In [274]:
with open("./data/user_factors.pkl", "r") as f:
    a = pickle.load(f)

# EXTRA


Running the cell to get all valid_zone_ids showed that all songs in data are valid songs in the MSD

In [130]:
song_ids

[u'SOVLGJY12A8C13FBED',
 u'SOGDQZK12A8C13F37C',
 u'SODMVJR12A6D4F985D',
 u'SOIWBDR12A8C13A4AC',
 u'SOHCCIA12AC907577F',
 u'SOBOAQC12A8C13E3E9',
 u'SOKVLHX12AB0187B39',
 u'SOMMSMW12A8C13FCCC',
 u'SODPNJR12A6D4FA52D',
 u'SOFFLLP12AB018ED52',
 u'SOZSPTD12A8C143BFD',
 u'SOHWWOM12AB0186627',
 u'SOZVLXO12AF72ABCDB',
 u'SOVBVJW12AAF3B5F77',
 u'SOINHKC12AB01836E0',
 u'SODKUVJ12AB0187A4A',
 u'SOMJNMD12AB0187F93',
 u'SOZNBMX12AB0182AD1',
 u'SOIRIWN12AB0183AE7',
 u'SOESNFW12A6D4F995F',
 u'SOGNFKZ12A8C145224',
 u'SOSJVGP12AB0185B95',
 u'SOHHGUG12A6D4F4E77',
 u'SOBIPBF12AB0184B0C',
 u'SONUTGA12AB0187F44',
 u'SOIGIUZ12A8C1337B1',
 u'SORRQLD12A8C139E8D',
 u'SOHHKOU12A81C224B2',
 u'SOWUAYB12A6D4FA1D4',
 u'SOYTQGO12A8C138C1B',
 u'SOOOBCE12A8C134019',
 u'SOROLCY12AB0182652',
 u'SOWBEBK12AB018306A',
 u'SOLIAYU12A6D4F9923',
 u'SOOBRTK12A8C13D6A9',
 u'SONROBV12AB0188BC7',
 u'SOLWBGS12AB0187EAD',
 u'SOOCWAO12AB01852C1',
 u'SOJDJRK12AB018565D',
 u'SOYNXWG12AC468F51A',
 u'SOQYKBE12AC46867EE',
 u'SOHWKQE12A8C1